In [ ]:
import torch

print(torch.__version__)

2.1.0+cu121


In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


#FAST-SAM
(SAM, SAM-HQ는 2024.01.03 현재 text prompt를 입력값으로 받지 않음, only point, only bounding box)
(그 앞에 grounding dino를 쓸 뿐)

In [ ]:
!git clone https://github.com/CASIA-IVA-Lab/FastSAM.git

In [ ]:
!wget https://huggingface.co/spaces/An-619/FastSAM/resolve/main/weights/FastSAM.pt

In [ ]:
!pip install -r FastSAM/requirements.txt
!pip install git+https://github.com/openai/CLIP.git

In [ ]:
!python FastSAM/Inference.py  --model_path FastSAM.pt --img_path ./images/dog.jpg --imgsz 1024

##한 이미지만 실행

In [ ]:
!wget -P images https://raw.githubusercontent.com/facebookresearch/segment-anything/main/notebooks/images/dog.jpg

In [ ]:
!python Inference.py --model_path ./weights/FastSAM.pt --img_path ./images/dogs.jpg  --text_prompt "the yellow dog"

## 특정 input 폴더에 있는 이미지(png) 전부 실행

#vitmatte


In [ ]:
# download codes and pre-trained weights

import os
!git clone https://github.com/hustvl/ViTMatte.git
os.chdir('/content/ViTMatte')

!pip install gdown
import gdown
url='https://drive.google.com/u/0/uc?id=12VKhSwE_miF9lWQQCgK7mv83rJIls3Xe'
file_name='ViTMatte_S_Com.pth'
gdown.download(url, file_name)

Cloning into 'ViTMatte'...
remote: Enumerating objects: 162, done.
remote: Counting objects: 100% (55/55), done.
remote: Compressing objects: 100% (17/17), done.
remote: Total 162 (delta 42), reused 38 (delta 38), pack-reused 107
Receiving objects: 100% (162/162), 3.96 MiB | 20.38 MiB/s, done.
Resolving deltas: 100% (66/66), done.
Access denied with the following error:



 	Too many users have viewed or downloaded this file recently. Please
	try accessing the file again later. If the file you are trying to
	access is particularly large or is shared with many people, it may
	take up to 24 hours to be able to view or download the file. If you
	still can't access a file after 24 hours, contact your domain
	administrator. 

You may still be able to access the file from the browser:

	 https://drive.google.com/u/0/uc?id=12VKhSwE_miF9lWQQCgK7mv83rJIls3Xe 



In [ ]:
# install packages

!pip install fairscale timm
!python -m pip install 'git+https://github.com/facebookresearch/detectron2.git'

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 266.3/266.3 kB 2.0 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for fairscale: filename=fairscale-0.4.13-py3-none-any.whl size=332104 sha256=4b70894570797edf96185e0d7a86553d31e763173c9f9a51a19c5c0df95cde06
  Stored in directory: /root/.cache/pip/wheels/78/a4/c0/fb0a7ef03cff161611c3fa40c6cf898f76e58ec421b88e8cb3
Successfully built fairscale
  Cloning https://github.com/facebookresearch/detectron2.git to /tmp/pip-req-build-bd7j1x3b
  Running command git clone --filter=blob:none --quiet https://github.com/facebookresearch/detectron2.git /tmp/pip-req-build-bd7j1x3b
  Resolved https://github.com/facebookresearch/detectron2.git to commit e9f7e2ba15abd7badcb05ef6f5076f06b36a9c5b
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.2/50.2 kB 1.2 MB/

In [ ]:
import os
import cv2
import torch
import numpy as np
#import gradio as gr
from PIL import Image
import torch
from torchvision.ops import box_convert
import sys

#from detectron2.config import LazyConfig, instantiate
from detectron2.config.lazy import LazyConfig
from detectron2.config.instantiate import instantiate
from detectron2.checkpoint import DetectionCheckpointer

In [ ]:
vitmatte_models = {
	'vit_b': '/content/drive/MyDrive/Matting-Anything/pretrained/ViTMatte_B_DIS.pth',
}

vitmatte_config = {
	'vit_b': '/content/drive/MyDrive/Matte-Anything/configs/matte_anything.py',
}

In [ ]:
from typing import Optional, Tuple

def generate_checkerboard_image(height, width, num_squares):
    num_squares_h = num_squares
    square_size_h = height // num_squares_h
    square_size_w = square_size_h
    num_squares_w = width // square_size_w


    new_height = num_squares_h * square_size_h
    new_width = num_squares_w * square_size_w
    image = np.zeros((new_height, new_width), dtype=np.uint8)

    for i in range(num_squares_h):
        for j in range(num_squares_w):
            start_x = j * square_size_w
            start_y = i * square_size_h
            color = 255 if (i + j) % 2 == 0 else 200
            image[start_y:start_y + square_size_h, start_x:start_x + square_size_w] = color

    image = cv2.resize(image, (width, height))
    image = cv2.cvtColor(image, cv2.COLOR_GRAY2RGB)

    return image

def init_vitmatte(model_type):
    """
    Initialize the vitmatte with model_type in ['vit_s', 'vit_b']
    """

    import sys
    sys.path.append("/usr/local/lib/python3.10/dist-packages/detectron2/config")

    cfg = LazyConfig.load(vitmatte_config[model_type])
    vitmatte = instantiate(cfg.model)
    vitmatte.to(device)
    vitmatte.eval()
    DetectionCheckpointer(vitmatte).load(vitmatte_models[model_type])

    return vitmatte

def generate_trimap(mask, erode_kernel_size=10, dilate_kernel_size=10):
    erode_kernel = np.ones((erode_kernel_size, erode_kernel_size), np.uint8)
    dilate_kernel = np.ones((dilate_kernel_size, dilate_kernel_size), np.uint8)
    eroded = cv2.erode(mask, erode_kernel, iterations=5)
    dilated = cv2.dilate(mask, dilate_kernel, iterations=5)
    trimap = np.zeros_like(mask)
    trimap[dilated==255] = 128
    trimap[eroded==255] = 255
    return trimap

def store_img(img):
    return img, []  # when new image is uploaded, `selected_points` should be empty

def convert_pixels(gray_image, boxes):
    converted_image = np.copy(gray_image)

    for box in boxes:
        x1, y1, x2, y2 = box
        x1, y1, x2, y2 = int(x1), int(y1), int(x2), int(y2)
        converted_image[y1:y2, x1:x2][converted_image[y1:y2, x1:x2] == 1] = 0.5

    return converted_image

def covert_tracer_output_to_vitmatte_input(image_path): #image path: '~~/~~/~~.png'
    print("image_path", image_path)
    # Load your background-removed image
    image = cv2.imread(image_path, cv2.IMREAD_UNCHANGED)

    # Assuming the alpha channel (if present) represents transparency (0 = removed, 255 = retained)
    if image.shape[2] == 4:  # Check if the image has an alpha channel
        alpha_channel = image[:, :, 3]
        mask = (alpha_channel == 0)  # Create a mask where removed part is True

        # Invert the mask if necessary (to have True for removed part and False for foreground)
        mask = ~mask

        # Convert the mask to a 1024x1024 list
        mask_list = mask.tolist()  # Convert the NumPy array to a list
        # Now mask_list contains the True/False values for removed/background parts

        return mask_list

import numpy as np
import torch
from PIL import Image
import torchvision.transforms.functional as F


#def cal_foreground(image_dir, alpha_dir):
def cal_foreground(image_dir, alpha):
    """
    Calculate the foreground of the image.
    Input:
        image_dir: the directory of the image
        alpha_dir: the directory of the alpha matte
    Output:
        foreground: the foreground of the image, numpy array
    """
    image = Image.open(image_dir).convert('RGB')
    #alpha = Image.open(alpha_dir).convert('L')
    alpha = alpha.convert('L')
    alpha = F.to_tensor(alpha).unsqueeze(0)
    image = F.to_tensor(image).unsqueeze(0)
    foreground = image * alpha + (1 - alpha)
    foreground = foreground.squeeze(0).permute(1, 2, 0).numpy()

    return foreground


def cal_foreground2(image_dir, alpha_np):
    """
    Calculate the foreground of the image with transparent background.
    Input:
        image_dir: the directory of the image
        alpha: the alpha mask as a PIL Image (1024, 1024)
    Output:
        foreground: the foreground of the image, PIL Image
    """
    image = Image.open(image_dir).convert('RGBA')

    # Create a new blank RGBA image with the same size as the original image
    foreground = Image.new('RGBA', image.size)

    # Set alpha values based on the provided alpha mask
    for y in range(image.size[1]):
        for x in range(image.size[0]):
            r, g, b, a_original = image.getpixel((x, y))  # Get RGBA values from the original image

            # Multiply alpha channel with the provided alpha mask
            a = int(a_original * alpha_np[y, x] / 255)  # Scale alpha values to [0, 255]

            # Set the RGBA values for each pixel in the new image
            foreground.putpixel((x, y), (r, g, b, a))

    return foreground


# common functions

import os
from PIL import Image
from os.path import join as opj
from torchvision.transforms import functional as F
from detectron2.engine import default_argument_parser
from detectron2.config import LazyConfig, instantiate
from detectron2.checkpoint import DetectionCheckpointer
from google.colab.patches import cv2_imshow
import matplotlib.pyplot as plt

def infer_one_image(model, input, save_dir=None):
    """
    Infer the alpha matte of one image.
    Input:
        model: the trained model
        image: the input image
        trimap: the input trimap
    """
    output = model(input)['phas'].flatten(0, 2)
    output = F.to_pil_image(output) #tensor to pil

    #output.save(opj(save_dir))
    #output.save(save_dir)


    #return None
    return output

def init_model(model, checkpoint, device):
    """
    Initialize the model.
    Input:
        config: the config file of the model
        checkpoint: the checkpoint of the model
    """
    assert model in ['vitmatte-s', 'vitmatte-b']
    if model == 'vitmatte-s':
        config = '/content/ViTMatte/configs/common/model.py'
        cfg = LazyConfig.load(config)
        model = instantiate(cfg.model)
        model.to(device)
        model.eval()
        DetectionCheckpointer(model).load(checkpoint)
    elif model == 'vitmatte-b':
        config = '/content/ViTMatte/configs/common/model.py'
        cfg = LazyConfig.load(config)
        cfg.model.backbone.embed_dim = 768
        cfg.model.backbone.num_heads = 12
        cfg.model.decoder.in_chans = 768
        model = instantiate(cfg.model)
        model.to(device)
        model.eval()
        DetectionCheckpointer(model).load(checkpoint)
    return model


def get_data(image_dir, trimap_dir):
#def get_data(image_dir, trimap):
    """
    Get the data of one image.
    Input:
        image_dir: the directory of the image
        trimap_dir: the directory of the trimap
    """
    image = Image.open(image_dir).convert('RGB')
    #image = Image.fromarray(image).convert('RGB')
    image = F.to_tensor(image).unsqueeze(0)
    trimap = Image.open(trimap_dir).convert('L')
    #trimap = Image.fromarray(trimap).convert('L')
    trimap = F.to_tensor(trimap).unsqueeze(0)

    return {
        'image': image,
        'trimap': trimap
    }





def merge_new_bg(image_dir, bg_dir, alpha_dir):
    """
    Merge the alpha matte with a new background.
    Input:
        image_dir: the directory of the image
        bg_dir: the directory of the new background
        alpha_dir: the directory of the alpha matte
    """
    image = Image.open(image_dir).convert('RGB')
    bg = Image.open(bg_dir).convert('RGB')
    alpha = Image.open(alpha_dir).convert('L')
    image = F.to_tensor(image)
    bg = F.to_tensor(bg)
    bg = F.resize(bg, image.shape[-2:])
    alpha = F.to_tensor(alpha)
    new_image = image * alpha + bg * (1 - alpha)

    new_image = new_image.squeeze(0).permute(1, 2, 0).numpy()
    return new_image


In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
import time

if torch.cuda.is_available():
    device = 'cuda'
else:
    device = 'cpu'

#vitmatte_model = 'vit_b'
vitmatte_model = init_model(model='vitmatte-b', checkpoint='/content/gdrive/MyDrive/Matte-Anything/pretrained/ViTMatte_B_DIS.pth', device=device)

colors = [(255, 0, 0), (0, 255, 0)]
markers = [1, 5] #marker type

print('Initializing models... Please wait...')

#predictor = init_segment_anything(sam_model)
#vitmatte = init_vitmatte(vitmatte_model)  # ->이거 나중에 내 깃헙 말고 다운받아서 가져오는 꼴로 하기
#grounding_dino =  dino_load_model(grounding_dino['config'], grounding_dino['weight'])#dino_load_model(grounding_dino['config'], grounding_dino['weight'])


 #   def run_vitmatte(vitmatte_input, selected_points, erode_kernel_size, dilate_kernel_size, fg_box_threshold, fg_text_threshold, fg_caption, tr_box_threshold, tr_text_threshold, tr_caption = "glass, lens, crystal, diamond, bubble, bulb, web, grid"):
def run_vitmatte(model, input_x,trimap_dir, masks, erode_kernel_size, dilate_kernel_size, fg_box_threshold, fg_text_threshold, fg_caption, tr_box_threshold, tr_text_threshold, tr_caption = "glass, lens, crystal, diamond, bubble, bulb, web, grid"):
    #set_image(input_x, "RGB")

    # generate alpha matte
    torch.cuda.empty_cache()
    mask = masks.astype(np.uint8)*255
    trimap = generate_trimap(mask, erode_kernel_size, dilate_kernel_size).astype(np.float32)
    indices = np.where(trimap == 255)
    trimap[trimap==128] = 0.5
    trimap[trimap==255] = 1  # 잘 만들어짐 #array

    cv2.imwrite(trimap_dir, trimap*255)

    #input = get_data(input_x,trimap*255) #255해줌
    input = get_data(input_x, trimap_dir) #save_dir = where to save alpha(mask)

    torch.cuda.empty_cache()

    alpha = infer_one_image(model, input) #alpha: PIL Image

    # get a green background
    image_array = cv2.imread(input_x)
    background = generate_checkerboard_image(image_array.shape[0], image_array.shape[1], 8)

    # calculate foreground with alpha blending
    #foreground_alpha = input_x * np.expand_dims(alpha, axis=2).repeat(3,2)/255 + background * (1 - np.expand_dims(alpha, axis=2).repeat(3,2))/255
    foreground_alpha = cv2.cvtColor(image_array, cv2.COLOR_RGBA2RGB) * np.expand_dims(np.array(alpha), axis=2).repeat(3,2)/255 + background * (1 - np.expand_dims(np.array(alpha), axis=2).repeat(3,2))/255

    #fg = cal_foreground(input_x, save_dir)
    fg=cal_foreground(input_x, alpha)

    # Convert alpha to a numpy array
    alpha_np = alpha.convert('L')  # Convert to grayscale
    alpha_np = np.array(alpha_np)  # Convert to numpy array

    a=time.time()
    fg2 = cal_foreground2(input_x, alpha_np)
    b=time.time()
    print("fg2 time per input", b-a)

    # calculate foreground with mask
    #foreground_mask = input_x * np.expand_dims(mask/255, axis=2).repeat(3,2)/255 + background * (1 - np.expand_dims(mask/255, axis=2).repeat(3,2))/255
    foreground_mask = cv2.cvtColor(image_array, cv2.COLOR_RGBA2RGB) * np.expand_dims(mask/255, axis=2).repeat(3,2)/255 + background * (1 - np.expand_dims(mask/255, axis=2).repeat(3,2))/255

    #foreground_alpha[foreground_alpha>1] = 1
    foreground_mask[foreground_mask>1] = 1

    # return img, mask_all
    trimap[trimap==1] == 0.999


    return  mask, alpha, foreground_mask, fg, fg2


Initializing models... Please wait...


In [ ]:



#사진 여러개에 대해서 실행하는 코드

import os
import matplotlib.image as mpimg

erode_kernel_size =10
dilate_kernel_size=10
fg_box_threshold=0.5
fg_text_threshold=0.5
#fg_caption="glass of water" #for glass of water
fg_caption="cat"
tr_box_threshold=0.5
tr_text_threshold=0.5
tr_caption= "glass, lens, crystal, diamond, bubble, bulb, web, grid"  #transparent 할것 같은 예시들

# Define input and mask directories
input_dir = '/content/gdrive/MyDrive/스모어톡/프리렌서/배경제거/sam_test/input'
mask_dir = '/content/gdrive/MyDrive/스모어톡/프리렌서/배경제거/sam_test/output/inspyrenet' #수정
!mkdir /content/gdrive/MyDrive/스모어톡/프리렌서/배경제거/sam_test/output/inspyrenet_vitmatte
final_dir = '/content/gdrive/MyDrive/스모어톡/프리렌서/배경제거/sam_test/output/inspyrenet_vitmatte'#수정
!mkdir /content/gdrive/MyDrive/스모어톡/프리렌서/배경제거/sam_test/output/inspyrenet_vitmatte_transp
final_dir2 = '/content/gdrive/MyDrive/스모어톡/프리렌서/배경제거/sam_test/output/inspyrenet_vitmatte_transp'#수정

# Get lists of files in both directories
input_files = os.listdir(input_dir)
mask_files = os.listdir(mask_dir)

!mkdir /content/ViTMatte/trimap

# Iterate through the files in the input directory
for input_file in input_files:
    # Ensure the file has the correct extension (e.g., PNG, JPG)
    if input_file.endswith('.png') or input_file.endswith('.jpg'):
        # Construct the full paths for input image and mask
        input_path = os.path.join(input_dir, input_file)
        mask_path = os.path.join(mask_dir, input_file)  # Assuming corresponding mask filenames are the same
        final_path = os.path.join(final_dir,input_file)
        final_path2 = os.path.join(final_dir2,input_file)

        # Process the images
        # Your code to perform the operations using input_path and mask_path goes here
        # Example:
        mask_list = covert_tracer_output_to_vitmatte_input(mask_path)
        trimap_dir = '/content/ViTMatte/trimap/trimap.png'
        mask, alpha, foreground_mask, foreground_alpha, foreground_alpha2 = run_vitmatte(vitmatte_model, input_path, trimap_dir, np.array(mask_list).astype('uint8'), erode_kernel_size, dilate_kernel_size, fg_box_threshold, fg_text_threshold, fg_caption, tr_box_threshold, tr_text_threshold, tr_caption)

        plt.imsave(final_path, np.array(foreground_alpha), format='png')    # Save as PNG format
        plt.imsave(final_path2, np.array(foreground_alpha2), format='png')



mkdir: cannot create directory ‘/content/gdrive/MyDrive/스모어톡/프리렌서/배경제거/sam_test/output/inspyrenet_vitmatte’: File exists
mkdir: cannot create directory ‘/content/gdrive/MyDrive/스모어톡/프리렌서/배경제거/sam_test/output/inspyrenet_vitmatte_transp’: File exists
mkdir: cannot create directory ‘/content/ViTMatte/trimap’: File exists
image_path /content/gdrive/MyDrive/스모어톡/프리렌서/배경제거/sam_test/output/inspyrenet/고양이-ai-generated-flamel (1) (1).png
fg2 time per input 4.536909341812134
image_path /content/gdrive/MyDrive/스모어톡/프리렌서/배경제거/sam_test/output/inspyrenet/강아지-ai-generated-flamel (1) (1).png
fg2 time per input 6.181604623794556
image_path /content/gdrive/MyDrive/스모어톡/프리렌서/배경제거/sam_test/output/inspyrenet/학생-ai-generated-flamel.png
fg2 time per input 4.633269309997559
image_path /content/gdrive/MyDrive/스모어톡/프리렌서/배경제거/sam_test/output/inspyrenet/햄버거-ai-generated-flamel.png
fg2 time per input 6.1103010